In [ ]:
import numpy as np
import imageio as io
import imageio
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
tf.__version__

### Camera response functions

In [ ]:
hdr=io.imread("pfstools_hdr_images/hdr_gallery/forest_path.hdr",format='HDR-FI')
hdr=np.array(hdr)

In [ ]:
def gen_blur(images, gamma, T, tau, nstd=2, seeds=None):
    '''
    Input a list of 3d tensors for images and kernels
    '''

    bsz, h, w, c = images.get_shape().as_list()
    tf.repeat(images,repeats=T+1,axis=0)
    for i in range(bsz):
        image=images[i]
        
    images = tf.transpose(images, [1,2,3,0]) # h x w x c x bsz
    power=tf.ones((images.shape[1],images.shape[2],images.shape[3],images.shape[0]))
    gamma=tf.constant(gamma,dtype=tf.float32)
    power=tf.multiply(power,gamma)
    
    dts=list(np.flip([1/(tau**i) for i in np.arange(1,T//2+1)]))
    dts=dts+[1]
    dts=dts+[tau**i for i in np.arange(1,T//2+1)]
    
    dts=tf.constant(dts,dtype=tf.float32)
    ldrs=tf.multiply(images,dts)
    tf.clip_by_value(ldrs, clip_value_min=0, clip_value_max=1)
    ldrs=tf.pow(ldrs,power)


### Forward function

In [ ]:
def forward(x,T,tau,j):
    LDRs=[]
    # defining delta t's
    dts=list(np.flip([1/(tau**i) for i in np.arange(1,T//2+1)]))
    dts=dts+[1]
    dts=dts+[tau**i for i in np.arange(1,T//2+1)]
    for i in range(T+1):
        LDRs.append(x*dts[i])
    # Normalizing LDRs
    c=0.5/np.mean(LDRs[T//2])
    for i in range(T+1):
        LDRs[i]=np.clip(LDRs[i]*c,0,1)
    
    return LDRs[j[0]],LDRs[j[1]]
    

In [ ]:
tau=np.sqrt(2)    
T=8
x=hdr
LDRs=[]
# defining delta t's
dts=list(np.flip([1/(tau**i) for i in np.arange(1,T//2+1)]))
dts=dts+[1]
dts=dts+[tau**i for i in np.arange(1,T//2+1)]
for i in range(T+1):
    LDRs.append(x*dts[i])
# Normalizing LDRs
c=0.5/np.mean(LDRs[T//2])
for i in range(T+1):
    LDRs[i]=LDRs[i]*c
LDRs=np.clip(np.array(LDRs),0,1)



In [ ]:
plt.figure()
plt.imshow(LDRs[5])
plt.show()

In [ ]:
np.max(LDRs[0])

In [ ]:
d=len(LDRs[8][np.where(LDRs[8]<0.01)])

In [ ]:
d/np.prod(hdr.shape)*100

In [ ]:
np.mean(hdr)